*   Nama : **Virgiawan Teguh Kusuma**
*   Email : **181240000833@unisnu.ac.id**
*   Domisili : **Kabupaten Jepara, Jawa Tengah**

In [1]:
# import library 
import zipfile, os
from google.colab import drive
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import numpy as np
from google.colab import files
import matplotlib.image as mpimg

In [2]:
# load google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# extract file masker.zip
maskerZip = '/content/drive/MyDrive/masker.zip'
maskerZipRef = zipfile.ZipFile(maskerZip, 'r')
maskerZipRef.extractall('/content')
maskerZipRef.close()

In [4]:
# list directory
os.listdir('/content/Face Mask Dataset')

['Test', 'Train', 'Validation']

In [5]:
# inisialisasi path test, train, validation
pathTest = '/content/Face Mask Dataset/Test'
pathTrain = '/content/Face Mask Dataset/Train'
pathVal = '/content/Face Mask Dataset/Validation'

In [6]:
# augmentasi gambar dengan ImageDataGenerator
trainDataGenerator = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range=0.2,
                    fill_mode='nearest')

testDataGenerator = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range=0.2,
                    fill_mode='nearest')

In [7]:
trainGenerator = trainDataGenerator.flow_from_directory(
    pathTrain,
    target_size=(150, 150),
    batch_size=128,
    class_mode='categorical')

valGenerator = testDataGenerator.flow_from_directory(
    pathVal,
    target_size=(150, 150),
    batch_size=128,
    class_mode='categorical')

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [8]:
# model sequential
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [9]:
# konversi model TFLiteConverter
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpeuwjpnoi/assets


In [10]:
# simpan model dengan format .tflite
with tf.io.gfile.GFile('masker.tflite', 'wb') as f:
  f.write(tflite_model)

In [34]:
# menggunakan callback agar pengujian berhenti jika mae mencapai target
# callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
class callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') < 0.9):
      self.model.stop_training = True
callback = callback()

In [35]:
# compile model
model.compile(loss='categorical_crossentropy', 
              optimizer=tf.optimizers.Adam(), 
              metrics=['accuracy']
              )

In [36]:
# train model dengan fit
history = model.fit(trainGenerator,
                    steps_per_epoch=25,
                    epochs=100, 
                    validation_data=valGenerator,
                    validation_steps=5,
                    verbose=2,
                    callbacks=[callback]
                    )

Epoch 1/100
25/25 - 170s - loss: 0.0633 - accuracy: 0.9803 - val_loss: 0.0339 - val_accuracy: 0.9891
Epoch 2/100
25/25 - 164s - loss: 0.0428 - accuracy: 0.9819 - val_loss: 0.0207 - val_accuracy: 0.9875
Epoch 3/100
25/25 - 170s - loss: 0.0244 - accuracy: 0.9919 - val_loss: 0.0153 - val_accuracy: 0.9937
Epoch 4/100
25/25 - 171s - loss: 0.0181 - accuracy: 0.9941 - val_loss: 0.0081 - val_accuracy: 0.9969
Epoch 5/100
25/25 - 170s - loss: 0.0288 - accuracy: 0.9891 - val_loss: 0.0185 - val_accuracy: 0.9906
Epoch 6/100
25/25 - 165s - loss: 0.0290 - accuracy: 0.9893 - val_loss: 0.0155 - val_accuracy: 0.9937
Epoch 7/100
25/25 - 169s - loss: 0.0196 - accuracy: 0.9941 - val_loss: 0.0165 - val_accuracy: 0.9937
Epoch 8/100
25/25 - 169s - loss: 0.0317 - accuracy: 0.9887 - val_loss: 0.0259 - val_accuracy: 0.9891
Epoch 9/100
25/25 - 172s - loss: 0.0280 - accuracy: 0.9894 - val_loss: 0.0315 - val_accuracy: 0.9844
Epoch 10/100
25/25 - 165s - loss: 0.0344 - accuracy: 0.9890 - val_loss: 0.0244 - val_accura

KeyboardInterrupt: ignored

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Akurasi Model')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss Model')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.show()

In [ ]:
# memprediksi gambar
%matplotlib inline

uploaded = files.upload()
for fn in uploaded.keys():
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=128)
  predIdxs = np.argmax(classes, axis=1)
  
  print(fn)
  print(predIdxs)
  if classes[0, 0] != 0:
    print('Pakai Masker')
  else:
    print('Tidak Pakai Masker')